In [9]:
from genericpath import isfile
import json
import os
if __name__ == '__main__':
    #os.sys.path.append('./pytorch_geometric/torch_geometric')
    os.sys.path.append('./src')
from src.model.model import MMGNet
from src.utils.config import Config
from utils import util
import torch
import argparse


In [14]:
def load_config():
    # load config file
    config = Config("./config/mmgnet.json")
    #print(config)
    if 'NAME' not in config:
        config_name = os.path.basename('./config/mmgnet.json')
        if len(config_name) > len('config_'):
            name = config_name[len('config_'):]
            name = os.path.splitext(name)[0]
            translation_table = dict.fromkeys(map(ord, '!@#$'), None)
            name = name.translate(translation_table)
            config['NAME'] = name            
    config.LOADBEST = ''
    config.MODE = 'train'
    config.exp = 'test'

    return config

In [16]:
config = load_config()
os.environ["CUDA_LAUNCH_BLOCKING"] = "0"
util.set_random_seed(config.SEED)

# if config.VERBOSE:
#     print(config)

model = MMGNet(config)
model


_NAME ['SGFN', 'Mmgnet']
MODEL_PATH /home/knuvi/hojun/CVPR2023-VLSAT/src/model/model.py
NAME Mmgnet
PATH ./config
multi_view_root /home/knuvi/hojun/CVPR2023-VLSAT/
VERBOSE False
SEED 2020
MAX_EPOCHES 60
LR 0.0001
W_DECAY False
AMSGRAD False
LR_SCHEDULE Cosine
GPU [0]
SAVE_INTERVAL 2000
VALID_INTERVAL 10
LOG_INTERVAL 100
LOG_IMG_INTERVAL 100
WORKERS 0
Batch_Size 16
update_2d False
EVAL False
_EDGE_BUILD_TYPE ['FC', 'KNN']
EDGE_BUILD_TYPE KNN
WEIGHTING True
dict:MODEL
  N_LAYERS 2
  USE_SPATIAL True
  WITH_BN False
  USE_GCN True
  USE_RGB False
  USE_NORMAL False
  use_2d_feats True
  USE_CONTEXT True
  USE_GCN_EDGE True
  USE_REL_LOSS True
  OBJ_PRED_FROM_GCN True
  _GCN_TYPE ['TRIP', 'EAN']
  GCN_TYPE EAN
  _ATTENTION ['fat']
  ATTENTION fat
  DROP_OUT_ATTEN 0.5
  multi_rel_outputs True
  feature_transform False
  point_feature_size 512
  edge_feature_size 256
  clip_feat_dim 512
  lambda_o 0.1
  DIM_ATTEN 256
  _WEIGHT_EDGE ['BG', 'DYNAMIC', 'OCCU', 'NONE']
  WEIGHT_EDGE DYNAMIC
  OB